Clark_Kent_Markdown_to_MDX.ipynb Notebook Summary
This notebook implements a tool that transforms standard Markdown content into interactive MDX (Markdown + JSX) with enhanced components. The transformation process follows these key steps:

Setup and Configuration: The notebook begins by importing necessary libraries and setting up configurations, including defining available MDX component types (interactive, layout, media, and data components).
Core Functionality:

Markdown Parser: Extracts structure and content from markdown files
Content Analysis: Analyzes markdown content to identify enhancement opportunities (intended to use Gemini AI but falls back to rule-based analysis)
MDX Transformation: Converts markdown to MDX by adding interactive components
Report Generation: Creates detailed reports on the transformation process


Component Types:

Interactive components (CodeBlock, Accordion, TableOfContents, etc.)
Layout components (TwoColumn, Grid, Sidebar, CalloutBox)
Media components (ImageZoom, VideoPlayer, AnimatedIllustration)
Data visualization components (DataTable, Chart)


File Processing: The notebook provides functions to process individual files or entire directories of markdown files.
Execution: When run, the notebook successfully processed 9 out of 9 markdown files, creating enhanced MDX versions with interactive components, despite encountering Gemini API authentication errors (falling back to rule-based analysis).

In [13]:
from google import genai

client = genai.Client(api_key="")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI uses data and algorithms to mimic human intelligence.



In [16]:
# Cell 1: Imports and Setup

import os
import re
import json
import glob
from pathlib import Path
import asyncio
from datetime import datetime
from typing import Dict, Any, List
from tqdm.notebook import tqdm
from rich.console import Console
from rich import print as rprint

# Import the Google Generative AI library
from google import genai

# Set up Gemini API
API_KEY = ""  # Ensure this is stored securely in production
client = genai.Client(api_key=API_KEY)
MODEL_NAME = "gemini-1.5-flash"  # You can change to other models as needed

# Set up console for rich output
console = Console()

# Define project constants
OUTPUT_DIR = f"MDX_Transformed_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output will be saved to: {OUTPUT_DIR}")

# Define MDX component definitions
MDX_COMPONENTS = {
    "interactive": {
        "CodeBlock": "Interactive code editor with syntax highlighting and execution",
        "Accordion": "Expandable/collapsible sections for progressive disclosure",
        "Tabs": "Tabbed interface for alternative approaches or examples",
        "TableOfContents": "Interactive table of contents linked to headers",
        "Slider": "Interactive slider for adjusting parameters in demos",
        "Quiz": "Interactive quiz component for testing knowledge"
    },
    "layout": {
        "TwoColumn": "Two-column layout for side-by-side content",
        "Grid": "Grid layout for organizing content in cards",
        "Sidebar": "Content with an interactive sidebar for navigation",
        "CalloutBox": "Highlighted box for important information"
    },
    "media": {
        "ImageZoom": "Image with zoom capability",
        "VideoPlayer": "Enhanced video player with chapters and annotations",
        "AudioPlayer": "Audio player with transcript",
        "AnimatedIllustration": "SVG animations that illustrate concepts"
    },
    "data": {
        "DataTable": "Interactive data table with sorting and filtering",
        "Chart": "Interactive data visualization"
    }
}

print("Environment configured and ready.")

Output will be saved to: MDX_Transformed_20250520_203251
Environment configured and ready.


In [17]:
# Cell 2: Markdown Parser Function

def parse_markdown(file_path: str) -> Dict[str, Any]:
    """Parse a markdown file and extract its structure and content."""
    console.print(f"Parsing markdown file: {file_path}")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extract front matter if exists (YAML between --- markers)
        frontmatter = {}
        content_without_frontmatter = content
        
        frontmatter_match = re.match(r'^---\s*\n(.*?)\n---\s*\n(.*)', content, re.DOTALL)
        if frontmatter_match:
            try:
                # Simple YAML parsing
                frontmatter_text = frontmatter_match.group(1)
                for line in frontmatter_text.split('\n'):
                    if ':' in line:
                        key, value = line.split(':', 1)
                        frontmatter[key.strip()] = value.strip()
                
                content_without_frontmatter = frontmatter_match.group(2)
            except Exception as e:
                console.print(f"[yellow]Warning: Error parsing frontmatter: {str(e)}[/yellow]")
        
        # Analyze content structure
        headers = re.findall(r'^(#{1,6})\s+(.+)$', content_without_frontmatter, re.MULTILINE)
        code_blocks = re.findall(r'```(\w*)\n(.*?)```', content_without_frontmatter, re.DOTALL)
        images = re.findall(r'!\[(.*?)\]\((.*?)\)', content_without_frontmatter)
        links = re.findall(r'(?<!!)\[(.*?)\]\((.*?)\)', content_without_frontmatter)
        lists = re.findall(r'((?:^\s*[-*+]\s+.+\n)+)', content_without_frontmatter, re.MULTILINE)
        tables = re.findall(r'^\|(.+)\|$\n^\|[-:\|\s]+\|$\n((?:^\|.+\|$\n)+)', content_without_frontmatter, re.MULTILINE)
        
        # Count paragraphs (text blocks separated by blank lines)
        paragraphs = re.split(r'\n\s*\n', content_without_frontmatter)
        paragraphs = [p.strip() for p in paragraphs if p.strip()]
        
        structure = {
            "headers": [(len(h[0]), h[1]) for h in headers],
            "code_blocks": [{"language": lang, "content": code} for lang, code in code_blocks],
            "images": [{"alt": alt, "src": src} for alt, src in images],
            "links": [{"text": text, "url": url} for text, url in links],
            "list_blocks": len(lists),
            "tables": len(tables),
            "paragraphs": len(paragraphs),
            "word_count": len(content_without_frontmatter.split())
        }
        
        return {
            "file_path": file_path,
            "file_name": os.path.basename(file_path),
            "frontmatter": frontmatter,
            "content": content,
            "content_without_frontmatter": content_without_frontmatter,
            "structure": structure
        }
    
    except Exception as e:
        console.print(f"[red]Error parsing markdown file {file_path}: {str(e)}[/red]")
        return {
            "file_path": file_path,
            "file_name": os.path.basename(file_path),
            "error": str(e),
            "content": "",
            "frontmatter": {},
            "structure": {}
        }

# Test the parser on a simple string
test_md = """---
title: Test
---

# Header

Text paragraph.

```python
print("hello")
```

![image](url)
"""

test_result = parse_markdown("path/to/test.md" if not os.path.exists("path/to/test.md") else "path/to/test.md")
print(f"Parser function defined and tested.")

Parsing markdown file: path/to/test.md

Error parsing markdown file path/to/test.md: [Errno 2] No such file or directory: 'path/to/test.md'

Parser function defined and tested.


In [18]:
# Cell 3: Content Analysis Functions

async def analyze_content_with_ai(markdown_data: Dict[str, Any]) -> Dict[str, Any]:
    """Use Gemini to analyze markdown content and suggest enhancements."""
    structure = markdown_data.get("structure", {})
    file_name = markdown_data.get("file_name", "")
    
    # Create a summary of the markdown structure
    structure_summary = {
        "headers_count": len(structure.get("headers", [])),
        "code_blocks_count": len(structure.get("code_blocks", [])),
        "code_languages": list(set(block.get("language", "") for block in structure.get("code_blocks", []) if block.get("language", ""))),
        "images_count": len(structure.get("images", [])),
        "lists_count": structure.get("list_blocks", 0),
        "tables_count": structure.get("tables", 0),
        "paragraphs_count": structure.get("paragraphs", 0),
        "word_count": structure.get("word_count", 0)
    }
    
    # Create a sample of the content (first 2000 characters)
    content_sample = markdown_data.get("content_without_frontmatter", "")[:2000]
    if len(markdown_data.get("content_without_frontmatter", "")) > 2000:
        content_sample += "..."
    
    # Create a prompt for Gemini
    prompt = """Analyze this markdown content and suggest how to transform it into immersive MDX components.

FILE NAME: {0}

MARKDOWN STRUCTURE SUMMARY:
- Headers: {1}
- Code Blocks: {2} (Languages: {3})
- Images: {4}
- Lists: {5}
- Tables: {6}
- Paragraphs: {7}
- Word Count: {8}

CONTENT SAMPLE:
{9}

AVAILABLE MDX COMPONENTS:
Interactive Components:
- CodeBlock: Interactive code editor with syntax highlighting and execution
- Accordion: Expandable/collapsible sections for progressive disclosure
- Tabs: Tabbed interface for showing alternative examples
- TableOfContents: Interactive table of contents linked to headers
- Diagram: Interactive diagrams with hover states
- Quiz: Interactive quiz component for testing knowledge

Layout Components:
- TwoColumn: Two-column layout for side-by-side content
- Grid: Grid layout for organizing content
- Sidebar: Sidebar navigation with main content
- CalloutBox: Highlighted box for important information

Media Components:
- ImageZoom: Image with zoom capability
- VideoPlayer: Enhanced video player with chapters
- DataTable: Interactive data table with sorting

OUTPUT FORMAT:
Provide a detailed JSON response with the following structure:
{{
  "interactive_elements": {{
    "code_blocks": {{"recommended": true/false, "reason": "explanation"}},
    "lists": {{"recommended": true/false, "reason": "explanation"}},
    "table_of_contents": {{"recommended": true/false, "reason": "explanation"}},
    "tables": {{"recommended": true/false, "reason": "explanation"}},
    "images": {{"recommended": true/false, "reason": "explanation"}},
    "layout": {{"recommended": true/false, "type": "TwoColumn/Grid/Sidebar", "reason": "explanation"}}
  }},
  "media_enhancements": {{
    "diagrams": {{"recommended": true/false, "reason": "explanation"}},
    "animations": {{"recommended": true/false, "reason": "explanation"}},
    "videos": {{"recommended": true/false, "reason": "explanation"}}
  }},
  "analysis": "Detailed analysis of the content and why certain components would enhance it",
  "special_suggestions": "Any special or creative suggestions for making this content more immersive"
}}
""".format(
        file_name,
        structure_summary["headers_count"],
        structure_summary["code_blocks_count"],
        ", ".join(structure_summary["code_languages"]) if structure_summary["code_languages"] else "None",
        structure_summary["images_count"],
        structure_summary["lists_count"],
        structure_summary["tables_count"],
        structure_summary["paragraphs_count"],
        structure_summary["word_count"],
        content_sample
    )
    
    try:
        # Call Gemini API for analysis
        console.print("  Using Gemini to analyze content and suggest enhancements...")
        response = client.generate_content(
            model=MODEL_NAME,
            contents=prompt
        )
        
        # Parse JSON response
        try:
            # Try to extract JSON from the response
            response_text = response.text
            
            # Look for JSON pattern in the response
            json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                analysis_result = json.loads(json_str)
            else:
                # Try parsing the whole response as JSON
                analysis_result = json.loads(response_text)
                
        except json.JSONDecodeError:
            # If JSON parsing fails, create a structured result from the text
            console.print("[yellow]Warning: Could not parse AI response as JSON. Using rule-based analysis instead.[/yellow]")
            # Fall back to rule-based analysis
            analysis_result = analyze_content_fallback(markdown_data)
        
        return analysis_result
    
    except Exception as e:
        console.print(f"[yellow]Error using Gemini for analysis: {str(e)}. Using rule-based analysis instead.[/yellow]")
        # Fall back to rule-based analysis
        return analyze_content_fallback(markdown_data)

def analyze_content_fallback(markdown_data: Dict[str, Any]) -> Dict[str, Any]:
    """Fallback function for content analysis using rule-based approach."""
    structure = markdown_data.get("structure", {})
    
    # Initialize results
    interactive_elements = {}
    media_enhancements = {}
    
    # Check for code blocks
    code_blocks = structure.get("code_blocks", [])
    if code_blocks:
        languages = [block.get("language", "").lower() for block in code_blocks]
        interactive_elements["code_blocks"] = {
            "recommended": True,
            "reason": f"Found {len(code_blocks)} code blocks that would benefit from syntax highlighting and interactive execution."
        }
    
    # Check for images
    images = structure.get("images", [])
    if images:
        interactive_elements["images"] = {
            "recommended": True,
            "reason": f"Found {len(images)} images that could be enhanced with zoom capability."
        }
    
    # Check for headers and recommend table of contents
    headers = structure.get("headers", [])
    if len(headers) > 3:
        interactive_elements["table_of_contents"] = {
            "recommended": True,
            "reason": f"Found {len(headers)} headers, which would benefit from a navigable table of contents."
        }
    
    # Check for list blocks and suggest accordions
    if structure.get("list_blocks", 0) > 0:
        interactive_elements["lists"] = {
            "recommended": True,
            "reason": f"Found {structure.get('list_blocks', 0)} list blocks that could be converted to expandable accordions."
        }
    
    # Check for tables and suggest interactive data tables
    if structure.get("tables", 0) > 0:
        interactive_elements["tables"] = {
            "recommended": True,
            "reason": f"Found {structure.get('tables', 0)} tables that could be enhanced with sorting and filtering."
        }
    
    # General layout suggestions
    word_count = structure.get("word_count", 0)
    if word_count > 1000:
        interactive_elements["layout"] = {
            "recommended": True,
            "type": "TwoColumn",
            "reason": f"Content is {word_count} words long, which would benefit from a two-column layout for better readability."
        }
    
    # Check for keywords in headers suggesting diagrams or animations
    header_text = " ".join([h[1].lower() for h in headers])
    if any(keyword in header_text for keyword in ["how", "process", "workflow", "architecture", "diagram"]):
        media_enhancements["diagrams"] = {
            "recommended": True,
            "reason": "Content discusses processes or architectures that would benefit from interactive diagrams."
        }
    
    # Put it all together
    result = {
        "interactive_elements": interactive_elements,
        "media_enhancements": media_enhancements,
        "analysis": "Analysis based on content structure and patterns.",
        "special_suggestions": "Consider adding interactive elements that engage readers and help explain complex concepts."
    }
    
    return result

# Test the analysis function with a simple example
test_markdown_data = {
    "file_name": "test.md",
    "structure": {
        "headers": [(1, "Title"), (2, "Section 1")],
        "code_blocks": [{"language": "python", "content": "print('hello')"}],
        "images": [{"alt": "test", "src": "test.png"}],
        "list_blocks": 2,
        "tables": 1,
        "paragraphs": 5,
        "word_count": 150
    },
    "content_without_frontmatter": "# Title\n\nTest content\n\n## Section 1\n\n```python\nprint('hello')\n```"
}

print("Analysis functions defined.")

Analysis functions defined.


In [19]:
# Cell 4: MDX Transformation Function

def transform_to_mdx(markdown_data: Dict[str, Any], analysis: Dict[str, Any]) -> Dict[str, Any]:
    """Transform markdown content to MDX with interactive components."""
    
    content = markdown_data.get("content", "")
    content_without_frontmatter = markdown_data.get("content_without_frontmatter", content)
    frontmatter = markdown_data.get("frontmatter", {})
    
    interactive_elements = analysis.get("interactive_elements", {})
    media_enhancements = analysis.get("media_enhancements", {})
    
    # Create imports section based on recommended components
    imports = []
    components_used = []
    
    # Add imports for code blocks
    if interactive_elements.get("code_blocks", {}).get("recommended", False):
        imports.append("import { CodeBlock } from '@/components/CodeBlock';")
        components_used.append("CodeBlock")
    
    # Add imports for accordions if lists are recommended for conversion
    if interactive_elements.get("lists", {}).get("recommended", False):
        imports.append("import { Accordion, AccordionItem } from '@/components/Accordion';")
        components_used.append("Accordion")
    
    # Add imports for table of contents
    if interactive_elements.get("table_of_contents", {}).get("recommended", False):
        imports.append("import { TableOfContents } from '@/components/TableOfContents';")
        components_used.append("TableOfContents")
    
    # Add imports for interactive tables
    if interactive_elements.get("tables", {}).get("recommended", False):
        imports.append("import { DataTable } from '@/components/DataTable';")
        components_used.append("DataTable")
    
    # Add imports for media components
    if interactive_elements.get("images", {}).get("recommended", False):
        imports.append("import { ImageZoom } from '@/components/ImageZoom';")
        components_used.append("ImageZoom")
    
    if media_enhancements.get("diagrams", {}).get("recommended", False):
        imports.append("import { Diagram } from '@/components/Diagram';")
        components_used.append("Diagram")
    
    # Add layout imports
    layout = interactive_elements.get("layout", {})
    if layout.get("recommended", False):
        layout_type = layout.get("type", "TwoColumn")
        if layout_type == "TwoColumn":
            imports.append("import { TwoColumn } from '@/components/TwoColumn';")
            components_used.append("TwoColumn")
        elif layout_type == "Grid":
            imports.append("import { Grid } from '@/components/Grid';")
            components_used.append("Grid")
        elif layout_type == "Sidebar":
            imports.append("import { Sidebar, SidebarContent, SidebarNav } from '@/components/Sidebar';")
            components_used.append("Sidebar")
    
    # Add CalloutBox for important information
    imports.append("import { CalloutBox } from '@/components/CalloutBox';")
    components_used.append("CalloutBox")
    
    # Convert code blocks to interactive CodeBlock components
    enhanced_content = content_without_frontmatter
    
    if interactive_elements.get("code_blocks", {}).get("recommended", False):
        code_block_pattern = r'```(\w*)\n(.*?)```'
        
        def code_block_replacer(match):
            lang = match.group(1) or "text"
            code = match.group(2)
            return f'<CodeBlock language="{lang}">\n```{lang}\n{code}```\n</CodeBlock>'
        
        enhanced_content = re.sub(code_block_pattern, code_block_replacer, enhanced_content, flags=re.DOTALL)
    
    # Convert images to ImageZoom components
    if interactive_elements.get("images", {}).get("recommended", False):
        image_pattern = r'!\[(.*?)\]\((.*?)\)'
        
        def image_replacer(match):
            alt = match.group(1)
            src = match.group(2)
            return f'<ImageZoom alt="{alt}" src="{src}" />'
        
        enhanced_content = re.sub(image_pattern, image_replacer, enhanced_content)
    
    # Add Table of Contents if recommended
    if interactive_elements.get("table_of_contents", {}).get("recommended", False):
        # Add TableOfContents after the first heading
        first_heading_pattern = r'^(#\s+.+?$)'
        enhanced_content = re.sub(
            first_heading_pattern, 
            r'\1\n\n<TableOfContents />\n', 
            enhanced_content, 
            count=1, 
            flags=re.MULTILINE
        )
    
    # Create enhanced frontmatter
    enhanced_frontmatter = {**frontmatter}
    
    # Add MDX-specific fields to frontmatter
    enhanced_frontmatter["interactive"] = True
    enhanced_frontmatter["components"] = components_used
    
    # Format frontmatter as YAML
    frontmatter_yaml = "---\n"
    for key, value in enhanced_frontmatter.items():
        if isinstance(value, list):
            frontmatter_yaml += f"{key}:\n"
            for item in value:
                frontmatter_yaml += f"  - {item}\n"
        elif isinstance(value, bool):
            frontmatter_yaml += f"{key}: {str(value).lower()}\n"
        else:
            frontmatter_yaml += f"{key}: {value}\n"
    frontmatter_yaml += "---\n\n"
    
    # Add imports after frontmatter
    imports_text = "\n".join(imports) + "\n\n"
    
    # Add an intro callout to demonstrate component usage
    callout_text = '\n<CalloutBox type="info" title="Interactive MDX Content">\nThis content has been enhanced with interactive MDX components to improve your reading experience.\n</CalloutBox>\n\n'
    
    # Find the position after the first header to insert the callout
    first_header_match = re.search(r'^#\s+.+?$', enhanced_content, re.MULTILINE)
    if first_header_match and interactive_elements.get("table_of_contents", {}).get("recommended", False):
        # If we have a TOC, place callout after it
        toc_pattern = r'<TableOfContents />(\n+)'
        enhanced_content = re.sub(toc_pattern, r'<TableOfContents />\1' + callout_text, enhanced_content, count=1)
    elif first_header_match:
        # Otherwise place it directly after the first header
        header_end = first_header_match.end()
        enhanced_content = enhanced_content[:header_end] + '\n' + callout_text + enhanced_content[header_end:]
    
    # Apply TwoColumn layout if recommended
    if layout.get("recommended", False) and layout.get("type", "") == "TwoColumn":
        # Find all level 2 headers and wrap their sections in TwoColumn
        h2_sections = re.findall(r'(## .+?)(?=\n## |\Z)', enhanced_content, re.DOTALL)
        
        if len(h2_sections) > 1:
            # Only apply TwoColumn if we have multiple sections
            new_content = ""
            for i, section in enumerate(h2_sections):
                if i % 2 == 0 and i+1 < len(h2_sections):
                    # Start a new TwoColumn for every pair of sections
                    new_content += f'<TwoColumn>\n<div>\n{section}\n</div>\n<div>\n{h2_sections[i+1]}\n</div>\n</TwoColumn>\n\n'
                elif i % 2 == 1:
                    # Skip odd indices as they're handled in the previous iteration
                    continue
                else:
                    # Handle any remaining section
                    new_content += section
            
            # Replace the content after the first heading (preserving intro and TOC)
            first_heading_end = first_header_match.end()
            intro_content = enhanced_content[:first_heading_end]
            
            # Check if we have TOC and callout
            toc_callout_match = re.search(r'\n\n<TableOfContents />.+?<\/CalloutBox>\n\n', enhanced_content[first_heading_end:], re.DOTALL)
            if toc_callout_match:
                intro_content += enhanced_content[first_heading_end:first_heading_end+toc_callout_match.end()]
                enhanced_content = intro_content + new_content
            else:
                # Check if we just have callout
                callout_match = re.search(r'\n<CalloutBox.+?<\/CalloutBox>\n\n', enhanced_content[first_heading_end:], re.DOTALL)
                if callout_match:
                    intro_content += enhanced_content[first_heading_end:first_heading_end+callout_match.end()]
                    enhanced_content = intro_content + new_content
                else:
                    enhanced_content = intro_content + new_content
    
    # Combine all parts into the final MDX content
    mdx_content = frontmatter_yaml + imports_text + enhanced_content
    
    interactive_element_keys = [
        key for key, value in interactive_elements.items() 
        if isinstance(value, dict) and value.get("recommended", False)
    ]
    
    media_enhancement_keys = [
        key for key, value in media_enhancements.items() 
        if isinstance(value, dict) and value.get("recommended", False)
    ]
    
    return {
        "mdx_content": mdx_content,
        "components_used": components_used,
        "interactive_elements": interactive_element_keys,
        "media_enhancements": media_enhancement_keys
    }

def analyze_transformation(original_markdown: Dict[str, Any], mdx_result: Dict[str, Any]) -> Dict[str, Any]:
    """Analyze the transformation from markdown to MDX."""
    
    original_content = original_markdown.get("content", "")
    mdx_content = mdx_result.get("mdx_content", "")
    
    # Calculate statistics
    original_lines = len(original_content.split("\n"))
    mdx_lines = len(mdx_content.split("\n"))
    
    original_words = len(original_content.split())
    mdx_words = len(mdx_content.split())
    
    components_used = mdx_result.get("components_used", [])
    interactive_elements = mdx_result.get("interactive_elements", [])
    media_enhancements = mdx_result.get("media_enhancements", [])
    
    # Create an analysis report
    analysis = {
        "original_lines": original_lines,
        "mdx_lines": mdx_lines,
        "line_change": mdx_lines - original_lines,
        "line_change_percentage": ((mdx_lines - original_lines) / original_lines * 100) if original_lines > 0 else 0,
        
        "original_words": original_words,
        "mdx_words": mdx_words,
        "word_change": mdx_words - original_words,
        "word_change_percentage": ((mdx_words - original_words) / original_words * 100) if original_words > 0 else 0,
        
        "components_used": components_used,
        "components_count": len(components_used),
        "interactive_elements": interactive_elements,
        "media_enhancements": media_enhancements,
    }
    
    return analysis

def save_mdx_file(mdx_content: str, output_path: str) -> str:
    """Save MDX content to a file."""
    try:
        # Ensure the directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Write the content
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(mdx_content)
        
        console.print(f"[green]MDX file saved to: {output_path}[/green]")
        return output_path
    
    except Exception as e:
        console.print(f"[red]Error saving MDX file to {output_path}: {str(e)}[/red]")
        return ""

print("Transformation functions defined.")

Transformation functions defined.


In [20]:
# Cell 5: Report Generation Functions

def create_analysis_report(file_name: str, markdown_data: Dict[str, Any], analysis: Dict[str, Any]) -> str:
    """Create a human-readable analysis report."""
    
    # Create the header
    report = f"# MDX Enhancement Analysis for {file_name}\n\n"
    
    # Add content structure summary
    report += f"## Content Structure\n\n"
    report += f"- **Headers**: {len(markdown_data['structure']['headers'])}\n"
    report += f"- **Code Blocks**: {len(markdown_data['structure']['code_blocks'])}\n"
    report += f"- **Images**: {len(markdown_data['structure']['images'])}\n"
    report += f"- **Lists**: {markdown_data['structure']['list_blocks']}\n"
    report += f"- **Tables**: {markdown_data['structure']['tables']}\n"
    report += f"- **Paragraphs**: {markdown_data['structure']['paragraphs']}\n"
    report += f"- **Word Count**: {markdown_data['structure']['word_count']}\n\n"
    
    # Add interactive elements recommendations
    report += f"## Recommended Interactive Elements\n\n"
    
    interactive_elements = analysis.get("interactive_elements", {})
    for key, value in interactive_elements.items():
        if isinstance(value, dict) and "reason" in value:
            recommended = value.get("recommended", False)
            reason = value.get("reason", "")
            status = "✅ Recommended" if recommended else "❌ Not recommended"
            report += f"- **{key}**: {status}\n  {reason}\n\n"
    
    # Add media enhancement recommendations
    report += f"## Recommended Media Enhancements\n\n"
    
    media_enhancements = analysis.get("media_enhancements", {})
    for key, value in media_enhancements.items():
        if isinstance(value, dict) and "reason" in value:
            recommended = value.get("recommended", False)
            reason = value.get("reason", "")
            status = "✅ Recommended" if recommended else "❌ Not recommended"
            report += f"- **{key}**: {status}\n  {reason}\n\n"
    
    # Add analysis summary
    if "analysis" in analysis:
        report += f"## Analysis Summary\n\n{analysis.get('analysis', '')}\n\n"
    
    # Add special suggestions
    if "special_suggestions" in analysis:
        report += f"## Special Suggestions\n\n{analysis.get('special_suggestions', '')}\n"
    
    return report

def create_transformation_report(file_name: str, transformation_analysis: Dict[str, Any]) -> str:
    """Create a human-readable transformation report."""
    
    # Create the header
    report = f"# MDX Transformation Report for {file_name}\n\n"
    
    # Add transformation metrics
    report += f"## Transformation Metrics\n\n"
    report += f"- **Original Lines**: {transformation_analysis['original_lines']}\n"
    report += f"- **MDX Lines**: {transformation_analysis['mdx_lines']}\n"
    report += f"- **Line Change**: {transformation_analysis['line_change']} ({transformation_analysis['line_change_percentage']:.1f}%)\n"
    report += f"- **Original Words**: {transformation_analysis['original_words']}\n"
    report += f"- **MDX Words**: {transformation_analysis['mdx_words']}\n"
    report += f"- **Word Change**: {transformation_analysis['word_change']} ({transformation_analysis['word_change_percentage']:.1f}%)\n\n"
    
    # Add components used
    report += f"## Components Used\n\n"
    for component in transformation_analysis['components_used']:
        report += f"- **{component}**\n"
    
    return report

def create_summary_report(results: List[Dict[str, Any]], output_dir: str) -> str:
    """Create a summary report of all transformations."""
    
    # Filter results
    successful = [r for r in results if r["success"]]
    failed = [r for r in results if not r["success"]]
    
    # Create the header
    report = "# MDX Transformation Summary\n\n"
    report += f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    
    # Add transformation results
    report += "## Transformation Results\n\n"
    report += f"- Total Files Processed: {len(results)}\n"
    report += f"- Successful Transformations: {len(successful)}\n"
    report += f"- Failed Transformations: {len(failed)}\n\n"
    
    # Add component usage statistics
    all_components_used = []
    for result in successful:
        all_components_used.extend(result.get("components_used", []))
    
    component_counts = {}
    for component in all_components_used:
        if component in component_counts:
            component_counts[component] += 1
        else:
            component_counts[component] = 1
    
    if component_counts:
        report += "## Component Usage\n\n"
        for component, count in sorted(component_counts.items(), key=lambda x: x[1], reverse=True):
            report += f"- **{component}**: {count} files\n"
        report += "\n"
    
    # List successful transformations
    report += "## Successful Transformations\n\n"
    report += "| # | File | Components | Word Count | MDX Output |\n"
    report += "|---|------|------------|------------|------------|\n"
    
    for i, result in enumerate(successful):
        file_name = result.get("file_name", "Unknown")
        components_count = len(result.get("components_used", []))
        components_list = ", ".join(result.get("components_used", [])[:3])
        if len(result.get("components_used", [])) > 3:
            components_list += f" +{len(result.get('components_used', [])) - 3} more"
        
        word_count = result.get("analysis", {}).get("mdx_words", 0)
        output_path = result.get("output_mdx_path", "")
        relative_output = os.path.relpath(output_path, output_dir) if output_path else ""
        
        report += f"| {i+1} | {file_name} | {components_list} | {word_count} | [{relative_output}](./{relative_output}) |\n"
    
    # List failed transformations
    if failed:
        report += "\n## Failed Transformations\n\n"
        report += "| # | File | Error |\n"
        report += "|---|------|-------|\n"
        
        for i, result in enumerate(failed):
            file_name = result.get("file_name", "Unknown")
            error = result.get("errors", ["Unknown error"])[0]
            
            report += f"| {i+1} | {file_name} | {error} |\n"
    
    # Add process description
    report += "\n## Process Description\n\n"
    report += "Each markdown file went through the following transformation process:\n\n"
    report += "1. **Content Analysis**: Analyzed the structure and content of the markdown file using Gemini AI\n"
    report += "2. **MDX Enhancement**: Identified opportunities for interactive components\n"
    report += "3. **MDX Transformation**: Converted markdown to MDX with interactive components\n"
    report += "4. **Transformation Analysis**: Evaluated the transformation quality and effectiveness\n\n"
    
    report += "Each file directory contains detailed logs of each step in the process.\n"
    
    return report

print("Report generation functions defined.")

Report generation functions defined.


In [21]:
# Cell 6: File Processing Functions

async def process_markdown_file(file_path: str, output_dir: str = OUTPUT_DIR) -> Dict[str, Any]:
    """Process a single markdown file into an interactive MDX file."""
    
    file_name = os.path.basename(file_path)
    file_stem = os.path.splitext(file_name)[0]
    
    console.print(f"\n[bold blue]Processing:[/bold blue] {file_path}")
    
    # Create a directory for this file's artifacts
    file_output_dir = os.path.join(output_dir, file_stem)
    os.makedirs(file_output_dir, exist_ok=True)
    
    results = {
        "file_path": file_path,
        "file_name": file_name,
        "success": False,
        "output_mdx_path": "",
        "components_used": [],
        "interactive_elements": [],
        "media_enhancements": [],
        "analysis": {},
        "errors": []
    }
    
    try:
        # STEP 1: Parse the markdown
        console.print(f"  Step 1: Parsing markdown content...")
        markdown_data = parse_markdown(file_path)
        
        if "error" in markdown_data and markdown_data["error"]:
            raise Exception(f"Failed to parse markdown: {markdown_data['error']}")
        
        # Save parsed markdown data
        parsed_data_path = os.path.join(file_output_dir, "01_parsed_markdown.json")
        with open(parsed_data_path, "w") as f:
            # Convert structure to a serializable format for images and code blocks
            serializable_data = {**markdown_data}
            serializable_data["structure"] = {
                "headers": serializable_data["structure"]["headers"],
                "code_blocks": [
                    {"language": block["language"], "content_length": len(block["content"])} 
                    for block in serializable_data["structure"]["code_blocks"]
                ],
                "images": [
                    {"alt": img["alt"], "src": img["src"]} 
                    for img in serializable_data["structure"]["images"]
                ],
                "links": serializable_data["structure"]["links"],
                "list_blocks": serializable_data["structure"]["list_blocks"],
                "tables": serializable_data["structure"]["tables"],
                "paragraphs": serializable_data["structure"]["paragraphs"],
                "word_count": serializable_data["structure"]["word_count"]
            }
            json.dump(serializable_data, f, indent=2)
        
        # STEP 2: Analyze content for MDX opportunities using Gemini AI
        console.print(f"  Step 2: Analyzing content for MDX enhancement opportunities...")
        analysis = await analyze_content_with_ai(markdown_data)
        
        # Save analysis
        analysis_path = os.path.join(file_output_dir, "02_mdx_analysis.json")
        with open(analysis_path, "w") as f:
            json.dump(analysis, f, indent=2)
        
        # Create a human-readable analysis report
        analysis_report = create_analysis_report(file_name, markdown_data, analysis)
        
        # Save human-readable analysis
        analysis_report_path = os.path.join(file_output_dir, "02_enhancement_analysis.md")
        with open(analysis_report_path, "w") as f:
            f.write(analysis_report)
        
        # STEP 3: Transform to MDX
        console.print(f"  Step 3: Transforming to interactive MDX...")
        mdx_result = transform_to_mdx(markdown_data, analysis)
        
        # Extract MDX content
        mdx_content = mdx_result.get("mdx_content", "")
        
        # Save MDX content
        mdx_output_path = os.path.join(file_output_dir, f"{file_stem}.mdx")
        save_mdx_file(mdx_content, mdx_output_path)
        
        # STEP 4: Analyze transformation
        console.print(f"  Step 4: Analyzing transformation results...")
        transformation_analysis = analyze_transformation(markdown_data, mdx_result)
        
        # Save transformation analysis
        transform_analysis_path = os.path.join(file_output_dir, "03_transformation_analysis.json")
        with open(transform_analysis_path, "w") as f:
            json.dump(transformation_analysis, f, indent=2)
        
        # Create a human-readable transformation report
        transform_report = create_transformation_report(file_name, transformation_analysis)
        
        # Save human-readable transformation report
        transform_report_path = os.path.join(file_output_dir, "03_transformation_report.md")
        with open(transform_report_path, "w") as f:
            f.write(transform_report)
        
        # Update results
        results.update({
            "success": True,
            "output_mdx_path": mdx_output_path,
            "components_used": mdx_result.get("components_used", []),
            "interactive_elements": mdx_result.get("interactive_elements", []),
            "media_enhancements": mdx_result.get("media_enhancements", []),
            "analysis": transformation_analysis
        })
        
        console.print(f"[bold green]✓ Completed:[/bold green] {file_path} -> {mdx_output_path}")
        
    except Exception as e:
        error_message = f"Error processing {file_path}: {str(e)}"
        console.print(f"[bold red]Error:[/bold red] {error_message}")
        results["errors"].append(error_message)
        
        # Save error log
        error_log_path = os.path.join(file_output_dir, "error_log.md")
        with open(error_log_path, "w") as f:
            f.write(f"# Error Log: {file_name}\n\n")
            f.write(f"## Error Message\n\n")
            f.write(f"{error_message}\n\n")
    
    return results

async def process_markdown_directory(directory_path: str) -> Dict[str, Any]:
    """Process all markdown files in a directory into interactive MDX files."""
    console.print(f"\n[bold]MDX Transformer: Processing directory {directory_path}[/bold]")
    
    # Find all markdown files in the directory
    markdown_files = glob.glob(f"{directory_path}/**/*.md", recursive=True)
    
    if not markdown_files:
        console.print(f"[bold yellow]No markdown files found in {directory_path}[/bold yellow]")
        return {
            "status": "completed",
            "files_processed": 0,
            "successful_transformations": 0,
            "failed_transformations": 0,
            "output_directory": OUTPUT_DIR,
            "results": []
        }
    
    console.print(f"[bold]Found {len(markdown_files)} markdown files to process[/bold]")
    console.print(f"[bold]Output directory: {OUTPUT_DIR}[/bold]")
    
    # Process each file
    results = []
    for file_path in tqdm(markdown_files, desc="Processing files"):
        result = await process_markdown_file(
            file_path=file_path,
            output_dir=OUTPUT_DIR
        )
        results.append(result)
    
    # Create summary report
    summary_report = create_summary_report(results, OUTPUT_DIR)
    
    # Save summary report
    summary_path = os.path.join(OUTPUT_DIR, "00_transformation_summary.md")
    with open(summary_path, "w") as f:
        f.write(summary_report)
    
    # Count successful and failed transformations
    successful = [r for r in results if r["success"]]
    failed = [r for r in results if not r["success"]]
    
    console.print(f"\n[bold green]Transformation process completed![/bold green]")
    console.print(f"Processed {len(results)} files")
    console.print(f"Successful transformations: {len(successful)}")
    console.print(f"Failed transformations: {len(failed)}")
    console.print(f"Summary available at: {summary_path}")
    
    return {
        "status": "completed",
        "files_processed": len(results),
        "successful_transformations": len(successful),
        "failed_transformations": len(failed),
        "output_directory": OUTPUT_DIR,
        "summary_file": summary_path,
        "results": results
    }

print("File processing functions defined.")

File processing functions defined.


In [22]:
# Cell 8: Process Markdown Directory

# Define directory to process
MARKDOWN_DIR = "mvp_launch_markdown"

# Check if directory exists
if os.path.exists(MARKDOWN_DIR):
    process_dir = input(f"Do you want to process all files in {MARKDOWN_DIR}? (y/n): ")
    
    if process_dir.lower() == "y":
        console.print(f"Starting to process all files in {MARKDOWN_DIR}...")
        results = await process_markdown_directory(MARKDOWN_DIR)
        
        if results["status"] == "completed":
            console.print("\n" + "=" * 70)
            console.print("MDX TRANSFORMATION COMPLETED")
            console.print("=" * 70)
            console.print(f"\nTransformed {results['successful_transformations']} of {results['files_processed']} files")
            console.print(f"All transformed files with detailed logs are available in: {results['output_directory']}")
            console.print(f"Summary file: {results['summary_file']}")
    else:
        console.print("Directory processing skipped.")
else:
    console.print(f"[bold yellow]Directory {MARKDOWN_DIR} not found.[/bold yellow]")
    console.print("If you want to process files in a different directory, please provide the path.")
    
    custom_dir = input("Enter directory path (or press Enter to skip): ")
    
    if custom_dir and os.path.exists(custom_dir):
        console.print(f"Starting to process all files in {custom_dir}...")
        results = await process_markdown_directory(custom_dir)
        
        if results["status"] == "completed":
            console.print("\n" + "=" * 70)
            console.print("MDX TRANSFORMATION COMPLETED")
            console.print("=" * 70)
            console.print(f"\nTransformed {results['successful_transformations']} of {results['files_processed']} files")
            console.print(f"All transformed files with detailed logs are available in: {results['output_directory']}")
            console.print(f"Summary file: {results['summary_file']}")
    else:
        console.print("No valid directory provided. Skipping directory processing.")

Do you want to process all files in mvp_launch_markdown? (y/n):  y


Starting to process all files in mvp_launch_markdown...

MDX Transformer: Processing directory mvp_launch_markdown

Found 9 markdown files to process

Output directory: MDX_Transformed_20250520_203251

...

MDX TRANSFORMATION COMPLETED

======================================================================

Transformed 9 of 9 files

All transformed files with detailed logs are available in: MDX_Transformed_20250520_203251

Summary file: MDX_Transformed_20250520_203251/00_transformation_summary.md

In [10]:
# Debug the ADK structure to find the correct way to specify models

from google.adk import agents, runners, tools, sessions
import inspect

# Check what's available in the agents module
print("Available attributes in agents module:")
for attr in dir(agents):
    if not attr.startswith('_'):  # Skip private attributes
        print(f"- {attr}")

# Look at Agent's parameters
print("\nAgent constructor parameters:")
try:
    sig = inspect.signature(agents.Agent.__init__)
    for param_name, param in sig.parameters.items():
        if param_name != 'self':
            print(f"- {param_name}: {param.annotation}")
except Exception as e:
    print(f"Error inspecting Agent: {e}")

# Check if there's a specific module for models
try:
    from google.adk import models
    print("\nFound models module. Available attributes:")
    for attr in dir(models):
        if not attr.startswith('_'):
            print(f"- {attr}")
except ImportError:
    print("\nNo direct 'models' module found")

# Check for example agents
print("\nTrying to create a simple agent:")
try:
    # Try with a simple string model name
    test_agent = agents.Agent(
        name="test_agent",
        model="gemini-pro",  # Try without a Model class
        description="Test agent",
        instruction="Simple test"
    )
    print("Success creating agent with string model name!")
except Exception as e:
    print(f"Error creating agent with string model name: {e}")

# Try to find how to import LiteLLM if it exists
try:
    from google.adk.models import lite_llm
    print("\nFound lite_llm. Available attributes:")
    for attr in dir(lite_llm):
        if not attr.startswith('_'):
            print(f"- {attr}")
except ImportError:
    print("\nNo lite_llm module found")

Available attributes in agents module:
- Agent
- BaseAgent
- LiveRequest
- LiveRequestQueue
- LlmAgent
- LoopAgent
- ParallelAgent
- RunConfig
- SequentialAgent
- active_streaming_tool
- base_agent
- callback_context
- invocation_context
- live_request_queue
- llm_agent
- loop_agent
- parallel_agent
- readonly_context
- run_config
- sequential_agent
- transcription_entry

Agent constructor parameters:
- data: Any

Found models module. Available attributes:
- BaseLlm
- Gemini
- LLMRegistry
- LlmRequest
- LlmResponse
- base_llm
- base_llm_connection
- gemini_llm_connection
- google_llm
- lite_llm
- llm_request
- llm_response
- regex
- registry

Trying to create a simple agent:
Success creating agent with string model name!

Found lite_llm. Available attributes:
- Any
- AsyncGenerator
- BaseLlm
- BaseModel
- ChatCompletionAssistantMessage
- ChatCompletionDeveloperMessage
- ChatCompletionImageUrlObject
- ChatCompletionMessageToolCall
- ChatCompletionTextObject
- ChatCompletionToolMessage
-

In [12]:
# Try different approaches to create an agent

from google.adk import agents, runners, tools, sessions
from google.genai import types

# Set up session service
session_service = sessions.InMemorySessionService()
APP_NAME = "test_app"

# Approach 1: Try with just a model name string
try:
    test_agent1 = agents.Agent(
        name="test_agent1",
        model="gemini-pro",  # Just a string
        description="Test agent with string model",
        instruction="You are a simple test agent. Respond with a short confirmation message."
    )
    print("Created agent with string model name")
except Exception as e:
    print(f"Error with string model name: {e}")


# Approach 3: Look for any model-related classes in google.adk
try:
    # Import all modules from google.adk to inspect
    import google.adk
    all_modules = dir(google.adk)
    model_related = [m for m in all_modules if 'model' in m.lower()]
    print(f"Model-related modules: {model_related}")
except Exception as e:
    print(f"Error inspecting modules: {e}")

# Try to run the first agent if it was created successfully
async def try_run_agent():
    if 'test_agent1' in locals() or 'test_agent1' in globals():
        test_session_id = "test_session"
        session_service.create_session(
            app_name=APP_NAME,
            user_id="test_user",
            session_id=test_session_id
        )
        
        runner = runners.Runner(
            agent=test_agent1,
            app_name=APP_NAME,
            session_service=session_service
        )
        
        test_message = "Please confirm that you're working correctly."
        test_content = types.Content(role='user', parts=[types.Part(text=test_message)])
        
        try:
            print("Attempting to run the agent...")
            response_text = ""
            async for event in runner.run_async(
                user_id="test_user",
                session_id=test_session_id,
                new_message=test_content
            ):
                if hasattr(event, 'is_final_response') and (
                    (callable(event.is_final_response) and event.is_final_response()) or
                    (not callable(event.is_final_response) and event.is_final_response)
                ):
                    if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts') and event.content.parts:
                        part = event.content.parts[0]
                        if hasattr(part, 'text'):
                            response_text = part.text
            
            print(f"Agent response: {response_text[:100]}...")
            return True
        except Exception as e:
            print(f"Error running agent: {e}")
            import traceback
            traceback.print_exc()
            return False
    else:
        print("No agent was successfully created to run")
        return False

# Run the test
await try_run_agent()

Created agent with string model name
Model-related modules: ['models']
Attempting to run the agent...
Error running agent: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}


Traceback (most recent call last):
  File "/var/folders/qw/3z0h6fq537194cm5xh_z1gcc0000gn/T/ipykernel_25004/1533077587.py", line 55, in try_run_agent
    async for event in runner.run_async(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/adk/runners.py", line 197, in run_async
    async for event in invocation_context.agent.run_async(invocation_context):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/adk/agents/base_agent.py", line 133, in run_async
    async for event in self._run_async_impl(ctx):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/adk/agents/llm_agent.py", line 246, in _run_async_impl
    async for event in self._llm_flow.run_async(ctx):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/adk/flows/llm_flows/base_llm_flow.py", line 243, in run_async
    async for event in self._run_one_step_async(invocation_context):
  File "/Users/fal

False

In [6]:
# Simple Test for Agent and Model Setup

# Create a very simple test agent
test_agent = Agent(
    name="test_agent",
    model=LiteLlm(MODEL_GEMINI_1_5_FLASH),  # Using the correct model name
    description="Test agent to verify model setup",
    instruction="You are a simple test agent. Respond with a short confirmation message."
)

# Create a test session
test_session_id = "test_session"
session_service.create_session(
    app_name=APP_NAME,
    user_id="test_user",
    session_id=test_session_id
)

# Create a runner for the test agent
test_runner = Runner(
    agent=test_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Define a simple test function
async def run_test_agent():
    test_message = "Please confirm that you're working correctly."
    test_content = types.Content(role='user', parts=[types.Part(text=test_message)])
    
    response_text = ""
    
    try:
        async for event in test_runner.run_async(
            user_id="test_user",
            session_id=test_session_id,
            new_message=test_content
        ):
            if hasattr(event, 'is_final_response') and (
                (callable(event.is_final_response) and event.is_final_response()) or
                (not callable(event.is_final_response) and event.is_final_response)
            ):
                if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts') and event.content.parts:
                    part = event.content.parts[0]
                    if hasattr(part, 'text'):
                        response_text = part.text
        
        console.print(f"[bold green]✓ Agent test successful![/bold green]")
        console.print(f"[bold]Response:[/bold] {response_text[:100]}...")
        return True
    except Exception as e:
        console.print(f"[bold red]✗ Agent test failed: {str(e)}[/bold red]")
        console.print("Error details:", str(e))
        import traceback
        traceback.print_exc()
        return False

# Run the test
print("Testing agent with Gemini API...")
is_setup_working = await run_test_agent()
print(f"ADK and model setup working: {is_setup_working}")

if not is_setup_working:
    print("There seems to be an issue with the ADK or API key setup.")
    print("Please check your environment configuration before proceeding.")
else:
    print("Setup verification successful. Ready to proceed with MDX transformer.")

Testing agent with Gemini API...


20:16:37 - LiteLLM:ERROR: vertex_llm_base.py:290 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information.
Traceback (most recent call last):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 286, in get_access_token
    _credentials, credential_project_id = self.load_auth(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 95, in load_auth
    creds, creds_project_id = google_auth.default(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/auth/_default.py", line 685, in default
    raise exceptions.DefaultCredentialsError(_CLOUD_SDK_MISSING_CREDENTIALS)
google.auth.exceptions.DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



✗ Agent test failed: litellm.APIConnectionError: Your default credentials were not found. To set up Application 
Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Traceback (most recent call last):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/main.py", line 511, in 
acompletion
    response = await init_response
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/gemini/vertex_and_googl
e_ai_studio_gemini.py", line 1290, in async_completion
    _auth_header, vertex_project = await self._ensure_access_token_async(
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 359, in _ensure_access_token_async
    raise e
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 354, in _ensure_access_token_async
    return await asyncify(self.get_access_token)(
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/litellm_core_utils/asyncify.py", line 
57, in wrapper
    return await anyio.to_thread.run_sync(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/anyio/to_thread.py", line 56, in 
run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 
2470, in run_sync_in_worker_thread
    return await future
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 967,
in run
    result = context.run(func, *args)
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 293, in get_access_token
    raise e
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 286, in get_access_token
    _credentials, credential_project_id = self.load_auth(
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 95, in load_auth
    creds, creds_project_id = google_auth.default(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/auth/_default.py", line 685, in 
default
    raise exceptions.DefaultCredentialsError(_CLOUD_SDK_MISSING_CREDENTIALS)
google.auth.exceptions.DefaultCredentialsError: Your default credentials were not found. To set up Application 
Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

Error details: litellm.APIConnectionError: Your default credentials were not found. To set up Application Default 
Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Traceback (most recent call last):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/main.py", line 511, in 
acompletion
    response = await init_response
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/gemini/vertex_and_googl
e_ai_studio_gemini.py", line 1290, in async_completion
    _auth_header, vertex_project = await self._ensure_access_token_async(
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 359, in _ensure_access_token_async
    raise e
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 354, in _ensure_access_token_async
    return await asyncify(self.get_access_token)(
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/litellm_core_utils/asyncify.py", line 
57, in wrapper
    return await anyio.to_thread.run_sync(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/anyio/to_thread.py", line 56, in 
run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 
2470, in run_sync_in_worker_thread
    return await future
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 967,
in run
    result = context.run(func, *args)
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 293, in get_access_token
    raise e
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 286, in get_access_token
    _credentials, credential_project_id = self.load_auth(
  File 
"/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", 
line 95, in load_auth
    creds, creds_project_id = google_auth.default(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/google/auth/_default.py", line 685, in 
default
    raise exceptions.DefaultCredentialsError(_CLOUD_SDK_MISSING_CREDENTIALS)
google.auth.exceptions.DefaultCredentialsError: Your default credentials were not found. To set up Application 
Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

ADK and model setup working: False
There seems to be an issue with the ADK or API key setup.
Please check your environment configuration before proceeding.


Traceback (most recent call last):
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/main.py", line 511, in acompletion
    response = await init_response
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/gemini/vertex_and_google_ai_studio_gemini.py", line 1290, in async_completion
    _auth_header, vertex_project = await self._ensure_access_token_async(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 359, in _ensure_access_token_async
    raise e
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/llms/vertex_ai/vertex_llm_base.py", line 354, in _ensure_access_token_async
    return await asyncify(self.get_access_token)(
  File "/Users/falcon/anaconda3/envs/google-adk/lib/python3.9/site-packages/litellm/litellm_core_utils/asyncify.py", line 57, in wrapper
    return await anyio.to_thread.run_sync